<h1><center> Chatbot </center></h1>

<h4> Connecting to our MySQL database and extracting the questions from the posts

In [20]:
import mysql.connector
from mysql.connector import Error
try:
    mySQLconnection = mysql.connector.connect(host='localhost',
                             database='CATCH_UP',
                             user='root',
                             password='krutika')
    sql_select_question_Query = "select question_id, q_description,q_category from question"
    sql_answer_Query = "select question_id, q_r_description from q_response"
    cursor = mySQLconnection .cursor()
    cursor.execute(sql_select_question_Query)
    records = list(cursor.fetchall())
    cursor.execute(sql_answer_Query)
    responses = list(cursor.fetchall())
    cursor.close()
   
except Error as e :
    print ("Error while connecting to MySQL", e)

In [21]:
questions_list = [list(i) for i in records] 

<h4> Cleaning the data

In [22]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def no_stop_words(question):
    return [word for word in question if word not in stopwords.words('english')]

# Spliting each question into its set of words
def getWordsInQuestions(questions):
    words_in_questions = []

    for line in questions:
            questionNum = line[0]
            # cleaning the question
            question = line[1].lower().replace('?', '').split(' ')
            cleanQuestion = no_stop_words(question)
            # category
            category = line[2]
            
            words_in_questions.append([questionNum, cleanQuestion, category])

    
    return words_in_questions

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/krutikaambavane/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
words_in_questions = getWordsInQuestions(questions_list)


<h4> Computing Jaccard Similarity

In [24]:
# Compute the Jaccard Similarity of two sets
def compute_jaccard_similarity(set_1, set_2):
    return len(set_1 & set_2) / len(set_1 | set_2)


In [25]:
THRESHOLD = 0.3  #setting threshold for matching similar questions


responseList = [list(r) for r in responses]
finalResponses = {}
for r in responseList:
    if r[0] not in finalResponses:
        finalResponses[r[0]] = []
    finalResponses[r[0]].append(r[1])


qlist = {}
for q in questions_list:
    qlist[q[0]] = q[1]
        
def findQuestion(quid):
    return qlist[quid]

def findSimilarQuestions(newQuestion):
    similarQuestions = []
        
    # cleaning the new question taken from the user
    newQ = newQuestion.lower().replace('?', '').split(' ')
    cleanQ = no_stop_words(newQ)
    
    similarQ = []
    for question in words_in_questions:
        if compute_jaccard_similarity(set(question[1]), set(cleanQ)) >= THRESHOLD:
            similarQ.append(question[0])
    
    QuesAnswers = {}
    for quid in similarQ:
        if quid in finalResponses:
            QuesAnswers[findQuestion(quid)] = finalResponses[quid]
        
    return QuesAnswers
        

In [26]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [29]:
import cmd
import json

JSON_FILE="/Users/krutikaambavane/Desktop/questions.json"

def loadJson():
    with open(JSON_FILE) as json_file:
        data = json.load(json_file)
        return data['section_tree']

class Content:
    
    section_tree = loadJson() # load all section info and faq in python dictionary
    sections = list(section_tree.keys()) # section names
    
    # used for printing final list to screen
    def pretty_print_names(self, secList) :
        
        ss = ''
        if isinstance(secList, list) :
            ss += color.BOLD + "Please choose a number. \n\n" + color.END
            for i in range(len(secList)) :
                ss += color.BLUE + "{}. {}\n".format(i+1, secList[i]) + color.END
            
            ss += color.BLUE + "N. Type a new question" + color.END
        else:
            ss += color.BLUE + secList + color.END
        
        ss += "\n" + color.BOLD + "\nType \"Back\" to go back one level" + color.END + "\n\n"
        return ss
        
            
    def construct_keys(self, keyList) :
        if len(keyList) == 0:
            return []
        
        finalVal = Content.section_tree[keyList[0]]
        for key in keyList[1:]:
            if len(finalVal) == 0:
                return []
            
            finalVal = finalVal[key]
        if isinstance(finalVal, dict):
            return list(finalVal.keys())
        
        return finalVal
    
    def getSectionName(self, keyList, indx) :
        if len(keyList) == 0:
            return Content.sections[indx]
        
        listVals = self.construct_keys(keyList)
        
        if len(listVals) == 0:
            return ''
        
        return listVals[indx]
    
    def printSection(self, keyList) :
        
        if len(keyList) == 0:
            return self.printTop()
        
        finalVal = self.construct_keys(keyList)
             
        if len(finalVal) == 0 :  
            print(color.RED + "No FAQs in this section. Please type your specific question." + color.END)
            return -1
            
        print(self.pretty_print_names(finalVal))
    
    def printTop(self):
        print(self.pretty_print_names(Content.sections))
        
      
        
class FAQs(cmd.Cmd):
    #intro = color.BOLD + 'Welcome to our FAQ section. How may I help you ? Please choose a number.\n\n' + color.END
    intro = ''
    cont = Content()
    #cont.printTop()
    prompt = ''
    file = None
    lastSections = []
    
    def printSection(self, args):
        try:
            if self.cont.printSection(self.lastSections) == -1:
                self.do_N(args)
        except:
            return
        
    def do_1(self, args):
        lastSection = self.cont.getSectionName(self.lastSections, 0)
        self.lastSections.append(lastSection)
        self.printSection(args)
        
    def do_2(self, args):
        lastSection = self.cont.getSectionName(self.lastSections, 1)
        self.lastSections.append(lastSection)
        self.printSection(args)
        
    def do_3(self, args):
        lastSection = self.cont.getSectionName(self.lastSections, 2)
        self.lastSections.append(lastSection)
        self.printSection(args)
        
    def do_4(self, args):
        lastSection = self.cont.getSectionName(self.lastSections, 3)
        self.lastSections.append(lastSection)
        self.printSection(args)
        
    def do_5(self, args):
        lastSection = self.cont.getSectionName(self.lastSections, 4)
        self.lastSections.append(lastSection)
        self.printSection(args)
        
    def do_6(self, args):
        lastSection = self.cont.getSectionName(self.lastSections, 5)
        self.lastSections.append(lastSection)
        self.printSection(args)
        
    def do_n(self, args):
        self.do_N(args)
    
    def do_N(self, args):
        ques = input(color.BLUE + "Please type your question :" + color.END)
        similars = findSimilarQuestions(ques)
        
        dashLines = ''
        for i in range(100):
            dashLines += '='
        if len(similars) > 0:
            print(color.BLUE + "Questions similar to your questions are :" + color.END)
            for ques in similars:
                print("\n{}".format(dashLines))
                print(color.BOLD + "\n Question : {}".format(ques) + color.END)
                ansList = similars[ques]
                for i in range(len(ansList)):
                    print(color.GREEN + "\n Answer {}. {}".format(i, ansList[i]) + color.END)
        else:
            lastSec = "/".join(self.lastSections)
            print(color.RED + "\n Sorry, no similar question found." + color.END)
            if len(lastSec) > 0:
                print("Head over to {} section for further info".format(lastSec))
        return
    
    def do_back(self, args) :
        '''
        Go back a section
        '''
        if len(self.lastSections) > 0:
            del self.lastSections[-1]
        
        #self.cont.printSection(self.lastSections)
        self.printSection(args)
    
    def do_end(self, args):
        print(color.CYAN + "Goodbye! See you soon." + color.END)
        return True
    
    def do_bye(self, args):
        self.do_end(args)
        
    def default(self, args):
        print(color.RED + "\n You have entered a wrong option. Please enter again.\n" + color.END)
        self.printSection(args)
        
    
if __name__ == '__main__':
    intro = color.BOLD + 'Welcome to our FAQ section. How may I help you ?\n\n' + color.END
    cont = Content()
    intro += cont.pretty_print_names(Content.sections)
    FAQs().cmdloop(intro=intro)
    


Welcome to our FAQ section. How may I help you ?

Please choose a number. 

1. Academic Guidance
2. Job Opportunities
3. Accomodation
4. Buy/Sell
5. Events
6. Dining & Entertainment
N. Type a new question

Type "Back" to go back one level


2
Please choose a number. 

1. On-Campus
2. Industry
N. Type a new question

Type "Back" to go back one level


2
Please choose a number. 

1. When can I start working FULL-TIME?
2. What do I do when I get a job?
3. How can I tell if my job is “related” to my field of study?
4. How many hours do I have to work to be considered “employed”?
N. Type a new question

Type "Back" to go back one level


4
Atleast 20 hours per week.

Type "Back" to go back one level


1
back
Atleast 20 hours per week.

Type "Back" to go back one level


back
Please choose a number. 

1. When can I start working FULL-TIME?
2. What do I do when I get a job?
3. How can I tell if my job is “related” to my field of study?
4. How many hours do I have to work to be considered “emp

TypeError: string indices must be integers

In [ ]:
import pickle

with open("chatbot.pkl", "wb") as file_handler:
    pickle.dump(*regr* "output coming from", file_handler)
with open("chatbot.pkl", "rb") as file_handler:
    loaded_pickle = pickle.load(file_handler)

______